In [1]:
# Import the necessary packages and import the html file. Open the file with Beautiful Soup
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
airline_data_url="https://github.com/fivethirtyeight/data/blob/master/airline-safety/airline-safety.csv"

# Make a GET request to fetch the raw HTML content
airline_content = requests.get(airline_data_url).text

# Parse the html content
airline_soup = BeautifulSoup(airline_content, "lxml")

In [2]:
airline_col_header = []
airline_headers = airline_soup.find_all("th")
for th in airline_headers:
    airline_col_header.append(th.text)
airline_col_header

['airline',
 'avail_seat_km_per_week',
 'incidents_85_99',
 'fatal_accidents_85_99',
 'fatalities_85_99',
 'incidents_00_14',
 'fatal_accidents_00_14',
 'fatalities_00_14']

In [3]:
airline_data = []
air_data = airline_soup.find_all("tr")
for tr in air_data:
    airline_data.append(tr.text)
airline_df = pd.DataFrame(airline_data)
airline_df.columns = ['Data']
airline_df = airline_df.Data.str.split("\n",expand=True)
airline_df = airline_df.drop([0, 1,10], axis = 1)
airline_df = airline_df.drop([0])
airline_df.columns = airline_col_header
airline_df.head()

,airline,avail_seat_km_per_week,incidents_85_99,fatal_accidents_85_99,fatalities_85_99,incidents_00_14,fatal_accidents_00_14,fatalities_00_14
1,Aer Lingus,320906734,2,0,0,0,0,0
2,Aeroflot*,1197672318,76,14,128,6,1,88
3,Aerolineas Argentinas,385803648,6,0,0,1,0,0
4,Aeromexico*,596871813,3,1,64,5,0,0
5,Air Canada,1865253802,2,0,0,2,0,0


In [4]:
airline_df.to_csv('airline_data.csv')

In [5]:
veh_url="https://www-fars.nhtsa.dot.gov/Main/index.aspx"

# Make a GET request to fetch the raw HTML content
veh_content = requests.get(veh_url).text

# Parse the html content
veh_soup = BeautifulSoup(veh_content, "lxml")

In [6]:
vehicle_header = []
veh_headers = veh_soup.find("tr", attrs={"class": "hdr01_home"})
for th in veh_headers:
    th = th.text
    th = th.strip()
    vehicle_header.append(th)
vehicle_header = vehicle_header[1:]
vehicle_header

['2018',
 '2017',
 '2016',
 '2015',
 '2014',
 '2013',
 '2012',
 '2011',
 '2010',
 '2009',
 '2008',
 '2007',
 '2006',
 '2005',
 '2004',
 '2003',
 '2002',
 '2001',
 '2000',
 '1999',
 '1998',
 '1997',
 '1996',
 '1995',
 '1994']

In [7]:
fatality_rate = []
fatality_data = veh_soup.find_all("td", attrs={"headers": re.compile("hdr21 hdrCol.*")})
for td in fatality_data:
    fatality_rate.append(td.text)
df_fr = pd.DataFrame(fatality_rate)
df_fr.columns = ['Fatalities per 100 Million Vehicle Miles Traveled']
df_fr['Year'] = vehicle_header
df_fr.head()

,Fatalities per 100 Million Vehicle Miles Traveled,Year
0,1.13,2018
1,1.17,2017
2,1.19,2016
3,1.15,2015
4,1.08,2014


In [8]:
df_fr.to_csv('car_fatality_data.csv')

In [9]:
url_total_rev="http://web.mit.edu/airlinedata/www/2019%2012%20Month%20Documents/Traffic%20and%20Capacity/System%20Total/Total%20System%20Revenue%20Passenger%20Miles.htm"

total_rev_html_content = requests.get(url_total_rev).text

soup_total_rev = BeautifulSoup(total_rev_html_content, "lxml")

In [10]:
total_rev_header = []
total_rev_headers = soup_total_rev.find_all("td", attrs={"class": "xl728262"})
for td in total_rev_headers:
    total_rev_header.append(td.text)

In [11]:
total_revenue = []
total_rev = soup_total_rev.find_all("td", attrs={"class": "xl778262"})
for td in total_rev:
    total_revenue.append(td.text)

In [12]:
df_tr = pd.DataFrame(total_revenue)
df_tr.columns = ['Total Revenue Passenger Miles']
df_tr = df_tr.iloc[114:139]
df_tr = df_tr['Total Revenue Passenger Miles'].str.split("\n",expand=True)
df_tr = df_tr.drop([0], axis = 1)
df_tr.columns = ['Total Revenue Passenger Miles']
df_tr = df_tr.reset_index()
df_tr = df_tr.drop(['index'], axis = 1)
df_tr.head()

,Total Revenue Passenger Miles
0,"557,234"
1,"598,203"
2,"622,234"
3,"635,795"
4,"670,779"


In [13]:
url_dom_rev="http://web.mit.edu/airlinedata/www/2019%2012%20Month%20Documents/Traffic%20and%20Capacity/Domestic/Domestic%20Revenue%20Passenger%20Miles.htm"

dom_rev_html_content = requests.get(url_dom_rev).text

soup_dom_rev = BeautifulSoup(dom_rev_html_content, "lxml")

In [14]:
dom_revenue = []
total_dom_rev = soup_dom_rev.find_all("td", attrs={"class": "xl774871"})
for td in total_dom_rev:
    dom_revenue.append(td.text)
df_dr = pd.DataFrame(dom_revenue)
df_dr.columns = ['Total Domestic Revenue Passenger Miles']
df_dr = df_dr.iloc[125:]
df_dr = df_dr['Total Domestic Revenue Passenger Miles'].str.split("\n",expand=True)
df_dr = df_dr.drop([0], axis = 1)
df_dr.columns = ['Total Domestic Revenue Passenger Miles']
df_tr['Total Domestic Revenue Passenger Miles'] = df_dr['Total Domestic Revenue Passenger Miles'].to_list()
df_tr.head()

,Total Revenue Passenger Miles,Total Domestic Revenue Passenger Miles
0,"557,234","402,776"
1,"598,203","436,647"
2,"622,234","453,023"
3,"635,795","463,618"
4,"670,779","489,655"


In [15]:
url_intl_rev="http://web.mit.edu/airlinedata/www/2019%2012%20Month%20Documents/Traffic%20and%20Capacity/International/International%20Revenue%20Passenger%20Miles.htm"

intl_rev_html_content = requests.get(url_intl_rev).text

soup_intl_rev = BeautifulSoup(intl_rev_html_content, "lxml")

In [16]:
intl_revenue = []
total_intl_rev = soup_intl_rev.find_all("td", attrs={"class": "xl775725"})
for td in total_intl_rev:
    intl_revenue.append(td.text)
df_ir = pd.DataFrame(intl_revenue)
df_ir.columns = ['Total International Revenue Passenger Miles']
df_ir = df_ir.iloc[125:]
df_ir = df_ir['Total International Revenue Passenger Miles'].str.split("\n",expand=True)
df_ir = df_ir.drop([0], axis = 1)
df_ir.columns = ['Total International Revenue Passenger Miles']
df_tr['Total International Revenue Passenger Miles'] = df_ir['Total International Revenue Passenger Miles'].to_list()
df_tr['Year'] = total_rev_header
df_tr.head()

,Total Revenue Passenger Miles,Total Domestic Revenue Passenger Miles,Total International Revenue Passenger Miles,Year
0,"557,234","402,776","151,032",1995
1,"598,203","436,647","156,796",1996
2,"622,234","453,023","164,848",1997
3,"635,795","463,618","166,962",1998
4,"670,779","489,655","175,316",1999


In [17]:
df_tr_area = pd.DataFrame(df_tr[['Total Revenue Passenger Miles','Year']])
df_tr_area.columns = ['Revenue','Year']
df_tr_area['Category'] = ['Total' for i in range(25)]

df_tr_temp = pd.DataFrame(df_tr[['Total Domestic Revenue Passenger Miles','Year']])
df_tr_temp.columns = ['Revenue','Year']
df_tr_temp['Category'] = ['Domestic' for i in range(25)]
df_tr_area = df_tr_area.append(df_tr_temp)

df_tr_temp = pd.DataFrame(df_tr[['Total International Revenue Passenger Miles','Year']])
df_tr_temp.columns = ['Revenue','Year']
df_tr_temp['Category'] = ['International' for i in range(25)]
df_tr_area = df_tr_area.append(df_tr_temp)

df_tr_area.sort_values(by=["Year","Category"]).head()

,Revenue,Year,Category
0,"402,776",1995,Domestic
0,"151,032",1995,International
0,"557,234",1995,Total
1,"436,647",1996,Domestic
1,"156,796",1996,International


In [18]:
df_tr.to_csv('total_rev_data.csv')
df_tr_area.to_csv('total_rev_data_area.csv')

In [19]:
us_url = "https://www.bts.gov/sites/bts.dot.gov/files/table_02_09_121819.xlsx"
us_df = pd.read_excel(us_url, skiprows=1)
us_df.head()

,Unnamed: 0,1960,1965,1970,1975,1980,1985,1990,1991,1992,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Total fatalities,499,261,146,124,1.0,526.0,39.000,50.000,33.000,...,52.000,2.000,0.000,0.000,9.000000,0.000000,0.000000,0.000000,0.000000,1.000000
1,Total seriously injured persons,N,N,107,81,19.0,30.0,29.000,26.000,22.000,...,26.000,17.000,21.000,18.000,9.000000,14.000000,24.000000,18.000000,19.000000,25.000000
2,Total accidents,90,83,55,37,19.0,21.0,24.000,26.000,18.000,...,30.000,30.000,33.000,26.000,23.000000,31.000000,29.000000,30.000000,32.000000,30.000000
3,Fatal accidents,17,9,8,3,1.0,7.0,6.000,4.000,4.000,...,2.000,1.000,0.000,0.000,2.000000,0.000000,0.000000,0.000000,0.000000,1.000000
4,Aircraft-miles (millions),1130,1536,2685,2478,2924.0,3631.0,4947.832,4824.824,5039.435,...,7465.598,7598.128,7713.557,7660.072,7672.511158,7690.949524,7822.203105,8016.961206,8154.971756,8473.725686


In [20]:
us_years = us_df.columns.tolist()[1:]
us_acc_rate = us_df.iloc[8].tolist()[1:]
us_fatal_rate = us_df.iloc[9].tolist()[1:]
us_miles_flown = us_df.iloc[4].tolist()[1:]

In [21]:
us_data = pd.DataFrame(us_df.columns.tolist()[1:], columns = ["Years"])
us_data['Accident_Rate'] = us_acc_rate
us_data['Fatality_Rate'] = us_fatal_rate
us_data['Miles_Flown'] = us_miles_flown
us_data['Miles_Flown'] = us_data['Miles_Flown']*1000000
us_data.head()

,Years,Accident_Rate,Fatality_Rate,Miles_Flown
0,1960,7.964602,1.504425,1.130000e+09
1,1965,5.403646,0.585938,1.536000e+09
2,1970,2.048417,0.297952,2.685000e+09
3,1975,1.493140,0.121065,2.478000e+09
4,1980,0.649795,0.034200,2.924000e+09


In [22]:
us_data.to_csv('us_accident_data.csv')

In [25]:
flights_url = 'https://www.bts.dot.gov/annual-passengers-all-us-scheduled-airline-flights-domestic-international-and-foreign-airline'

flights_html_content = requests.get(flights_url).text

soup_flights = BeautifulSoup(flights_html_content, "lxml")

flights_table = soup_flights.find_all("table")[0]

In [26]:
years = []
year_data = flights_table.find_all('th')
for row in year_data:
    years.append(row.text)
years = years[5:]

flights_table = soup_flights.find_all("table")[0]
flight_results = []
rows = flights_table.find_all('tr')
for row in rows:
    data = row.find_all('td')
    row_data = [ x.text for x in data ]
    flight_results.append(row_data)

flight_results = pd.DataFrame(flight_results, columns = ['Total', 'Domestic', 'International','One','Two'])
flight_results = flight_results.drop(['One','Two'], axis=1)
flight_results = flight_results.drop([0, 17])
flight_results['Year'] = years
flight_results.head()

,Total,Domestic,International,Year
1,700.9,583.3,117.6,2003
2,763.7,629.8,133.9,2004
3,800.8,657.3,143.6,2005
4,808.1,658.4,149.7,2006
5,835.5,679.2,156.3,2007


In [27]:
flight_results.to_csv('flight_totals.csv')